# Data Collection
## Importing BTC Historic Data using Yahoo finance API

In [1]:
import pandas as pd

In [2]:
from datetime import datetime
import time
import json
import requests
import os
import serpapi
from dotenv import load_dotenv

In [1]:
def construct_download_url(
	ticker,
	period1,
	period2,
	interval='monthly'
):
	"""
	:period1 & period2: 'yyyy-mm-dd'
	:interval: {daily; weekly, monthly}
	"""
	def convert_to_seconds(period):
		datetime_value = datetime.strptime(period, '%Y-%m-%d')
		total_seconds = int(time.mktime(datetime_value.timetuple())) + 86400
		return total_seconds
	try:
		interval_reference = {'daily': '1d', 'weekly': '1wk', 'monthly': '1mo'}
		_interval = interval_reference.get(interval)
		if _interval is None:
			print('interval code is incorrect')
			return
		p1 = convert_to_seconds(period1)
		p2 = convert_to_seconds(period2)
		url = f'https://query1.finance.yahoo.com/v7/finance/download/{ticker}?period1={p1}&period2={p2}&interval={_interval}&filter=history'
		return url
	except Exception as e:
		print(e)
		return

# retrive dataset
query_url = construct_download_url('BTC-USD', '2021-01-01', '2022-06-30', 'daily')
df = pd.read_csv(query_url)
df.set_index('Date', inplace=True)

# save dataset as a CSV
df.to_csv('Bitcoin price 2021-2022.csv')

# save dataset as a JSON file
with open('BTC-USD.json', 'w') as f:
	f.write(json.dumps(df.T.to_dict(), indent=4))

## Importing Fear and Greed index data

In [ ]:

def fetch_fear_and_greed_index(limit=1, format='json', date_format='us'):
    # Base URL for the Fear and Greed Index API
    base_url = "https://api.alternative.me/fng/"

    # Construct the URL with optional parameters
    url = base_url + f"?limit={limit}&format={format}&date_format={date_format}"

    # Send GET request to the API
    response = requests.get(url)
    
    # Check if request was successful
    if response.status_code == 200:
        data = response.json()

        # Extracting historical data
        historical_data = data['data']

        # Convert data into pandas DataFrame
        df = pd.DataFrame(historical_data, columns=['value', 'value_classification', 'timestamp'])
        
        # Convert timestamp to datetime
        df['timestamp'] = pd.to_datetime(df['timestamp'], format='%m-%d-%Y')
        
        # Set timestamp as index
        df.set_index('timestamp', inplace=True)

        return df
    else:
        print(f"Failed to fetch data. Status code: {response.status_code}")
        return None

if __name__ == "__main__":
    # Example usage
    limit = 0  # Number of results to fetch
    format = 'json'  # Data format (json or csv)
    date_format = 'us'  # Date format (us, cn, kr, or world)
    
    fear_and_greed_index_data = fetch_fear_and_greed_index(limit=limit, format=format, date_format=date_format)
    
    if fear_and_greed_index_data is not None:
        # Export DataFrame to CSV
        fear_and_greed_index_data.to_csv('fear_and_greed_index_data.csv')


In [ ]:
fear_and_greed_index_data

## Importing Google Trends Data 

In [ ]:
import os
import serpapi
import pandas as pd
import json
from datetime import datetime, timedelta
from dotenv import load_dotenv

In [72]:
# Load environment variables
load_dotenv()

# Get SerpApi API key from environment variables
api_key = os.getenv('SERPAPI_KEY')

# Create a SerpApi client
client = serpapi.Client(api_key=api_key)

# Define the search parameters function
def search_google_trends(start_date, end_date):
    params = {
        'engine': 'google_trends',
        'q': 'bitcoin',
        'api_key': api_key,
        'geo': '',  # Worldwide
        'date': f'{start_date} {end_date}',
        'tz': '420',
        'data_type': 'TIMESERIES',
        'interval': 'daily'  # Ensure daily data
    }
    return client.search(**params)

# Define start and end dates for each 8-month period from 2016 to 2024
start_year = 2016
end_year = 2024
eight_months = timedelta(days=8*30)

# Create lists to store the extracted data
dates = []
timestamps = []
values = []

# Iterate over each 8-month period
current_date = datetime(start_year, 1, 1)
while current_date.year < end_year:
    start_date = current_date.strftime('%Y-%m-%d')
    end_date = (current_date + eight_months).strftime('%Y-%m-%d')
    
    # Perform the search
    search_result = search_google_trends(start_date, end_date)
    timeline_data = search_result['interest_over_time']['timeline_data']
    
    # Extract data from the search result
    for entry in timeline_data:
        date = entry['date']
        timestamp = entry['timestamp']
        for value_entry in entry['values']:
            value = value_entry['value']
            # Append data to lists
            dates.append(date)
            timestamps.append(timestamp)
            values.append(value)
    
    # Move to the next 8-month period
    current_date += eight_months

# Create a dictionary to store the extracted data
data_dict = {
    'date': dates,
    'timestamp': timestamps,
    'value': values
}

# Create a pandas DataFrame from the extracted data
df = pd.DataFrame(data_dict)

# Convert the 'timestamp' column to datetime format
df['timestamp'] = pd.to_datetime(df['timestamp'].astype(int), unit='s')

# Print the DataFrame
print(df)


              date  timestamp value
0      Jan 1, 2016 2016-01-01    29
1      Jan 2, 2016 2016-01-02    31
2      Jan 3, 2016 2016-01-03    33
3      Jan 4, 2016 2016-01-04    33
4      Jan 5, 2016 2016-01-05    36
...            ...        ...   ...
3017  Mar 24, 2024 2024-03-24    40
3018  Mar 25, 2024 2024-03-25    52
3019  Mar 26, 2024 2024-03-26    49
3020  Mar 27, 2024 2024-03-27    46
3021  Mar 28, 2024 2024-03-28    43

[3022 rows x 3 columns]


In [73]:
df.to_csv('bitcoin_gt.csv')

In [74]:
import os
import serpapi
import pandas as pd
import json
from datetime import datetime, timedelta
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Get SerpApi API key from environment variables
api_key = os.getenv('SERPAPI_KEY')

# Create a SerpApi client
client = serpapi.Client(api_key=api_key)

# Define the search parameters function
def search_google_trends(start_date, end_date):
    params = {
        'engine': 'google_trends',
        'q': 'crypto',
        'api_key': api_key,
        'geo': '',  # Worldwide
        'date': f'{start_date} {end_date}',
        'tz': '420',
        'data_type': 'TIMESERIES',
        'interval': 'daily'  # Ensure daily data
    }
    return client.search(**params)

# Define start and end dates for each 8-month period from 2016 to 2024
start_year = 2016
end_year = 2024
eight_months = timedelta(days=8*30)

# Create lists to store the extracted data
dates = []
timestamps = []
values = []

# Iterate over each 8-month period
current_date = datetime(start_year, 1, 1)
while current_date.year < end_year:
    start_date = current_date.strftime('%Y-%m-%d')
    end_date = (current_date + eight_months).strftime('%Y-%m-%d')
    
    # Perform the search
    search_result = search_google_trends(start_date, end_date)
    timeline_data = search_result['interest_over_time']['timeline_data']
    
    # Extract data from the search result
    for entry in timeline_data:
        date = entry['date']
        timestamp = entry['timestamp']
        for value_entry in entry['values']:
            value = value_entry['value']
            # Append data to lists
            dates.append(date)
            timestamps.append(timestamp)
            values.append(value)
    
    # Move to the next 8-month period
    current_date += eight_months

# Create a dictionary to store the extracted data
data_dict = {
    'date': dates,
    'timestamp': timestamps,
    'value': values
}

# Create a pandas DataFrame from the extracted data
df2 = pd.DataFrame(data_dict)

# Convert the 'timestamp' column to datetime format
df2['timestamp'] = pd.to_datetime(df2['timestamp'].astype(int), unit='s')

# Print the DataFrame
print(df2)


              date  timestamp value
0      Jan 1, 2016 2016-01-01    36
1      Jan 2, 2016 2016-01-02    48
2      Jan 3, 2016 2016-01-03    48
3      Jan 4, 2016 2016-01-04    59
4      Jan 5, 2016 2016-01-05    54
...            ...        ...   ...
3017  Mar 24, 2024 2024-03-24    55
3018  Mar 25, 2024 2024-03-25    65
3019  Mar 26, 2024 2024-03-26    66
3020  Mar 27, 2024 2024-03-27    64
3021  Mar 28, 2024 2024-03-28    66

[3022 rows x 3 columns]


In [75]:
df2.to_csv('crypto_gt.csv')

In [ ]:
import tweepy
import pandas as pd
from textblob import TextBlob
from datetime import datetime, timedelta

# Twitter API credentials
consumer_key = 'your_consumer_key'
consumer_secret = 'your_consumer_secret'
access_token = 'your_access_token'
access_token_secret = 'your_access_token_secret'

# Authenticate with Twitter API
auth = tweepy.OAuth1UserHandler(consumer_key, consumer_secret, access_token, access_token_secret)
api = tweepy.API(auth)

# Function to collect tweets and calculate sentiment
def collect_tweets_sentiment(keywords, start_date, end_date, num_tweets):
    tweets_sentiment = pd.DataFrame(columns=['Date', 'Sentiment'])

    # Iterate over each day
    current_date = start_date
    while current_date <= end_date:
        total_polarity = 0
        total_subjectivity = 0
        num_collected_tweets = 0

        # Collect tweets for each keyword
        for keyword in keywords:
            query = keyword + ' -filter:retweets'
            tweets = tweepy.Cursor(api.search, q=query, lang='en', since=current_date, until=current_date, tweet_mode='extended').items(num_tweets)
            
            # Analyze sentiment of collected tweets
            for tweet in tweets:
                text = tweet.full_text
                analysis = TextBlob(text)
                polarity = analysis.sentiment.polarity
                total_polarity += polarity
                total_subjectivity += analysis.sentiment.subjectivity
                num_collected_tweets += 1
        
        # Calculate average sentiment for the day
        if num_collected_tweets > 0:
            average_polarity = total_polarity / num_collected_tweets
            average_subjectivity = total_subjectivity / num_collected_tweets
            tweets_sentiment = tweets_sentiment.append({'Date': current_date, 'Sentiment': average_polarity}, ignore_index=True)

        # Move to the next day
        current_date += timedelta(days=1)

    return tweets_sentiment

# Set parameters
keywords = ['bitcoin', 'crypto']
start_date = datetime(2018, 1, 1)
end_date = datetime(2024, 1, 1)
num_tweets = 5000

# Collect tweets and calculate sentiment
tweets_sentiment = collect_tweets_sentiment(keywords, start_date, end_date, num_tweets)

# Store results in a CSV file
tweets_sentiment.to_csv('tweets_sentiment.csv', index=False)


## Filtering data from 2021-2022

In [27]:
fng = pd.read_csv('fear_and_greed_index_data.csv')

In [28]:
fng

,timestamp,value,value_classification
0,2024-03-27,83,Extreme Greed
1,2024-03-26,81,Extreme Greed
2,2024-03-25,75,Greed
3,2024-03-24,74,Greed
4,2024-03-23,73,Greed
...,...,...,...
2239,2018-02-05,11,Extreme Fear
2240,2018-02-04,24,Extreme Fear
2241,2018-02-03,40,Fear
2242,2018-02-02,15,Extreme Fear


In [29]:
# Filter the DataFrame to include only data between January 1, 2021, and June 30, 2022
filtered_data = fng[(fng['timestamp'] <= '2022-06-30') & (fng['timestamp'] >= '2021-01-01')]


# Reset the index of the resulting DataFrame if needed
filtered_data.reset_index(drop=True, inplace=True)

filtered_data = filtered_data.sort_values(by='timestamp').reset_index(drop=True)


In [30]:
filtered_data.to_csv('FnG_index_21_22.csv')

In [31]:
btc_gt = pd.read_csv('bitcoin_gt.csv')

In [32]:
btc_gt.drop(columns=['Unnamed: 0'], inplace=True)

In [46]:
btc_gt

,date,timestamp,value
0,"Jan 1, 2016",2016-01-01,29
1,"Jan 2, 2016",2016-01-02,31
2,"Jan 3, 2016",2016-01-03,33
3,"Jan 4, 2016",2016-01-04,33
4,"Jan 5, 2016",2016-01-05,36
...,...,...,...
3017,"Mar 24, 2024",2024-03-24,40
3018,"Mar 25, 2024",2024-03-25,52
3019,"Mar 26, 2024",2024-03-26,49
3020,"Mar 27, 2024",2024-03-27,46


In [33]:
# Filter the DataFrame to include only data between January 1, 2021, and June 30, 2022
gt_filtered = btc_gt[(btc_gt['timestamp'] <= '2022-06-30') & (btc_gt['timestamp'] >= '2021-01-01')]


# Reset the index of the resulting DataFrame if needed
gt_filtered.reset_index(drop=True, inplace=True)

gt_filtered = gt_filtered.sort_values(by='timestamp').reset_index(drop=True)

In [48]:
gt_filtered

,date,timestamp,value
0,"Jan 1, 2021",2021-01-01,39
1,"Jan 2, 2021",2021-01-02,64
2,"Jan 3, 2021",2021-01-03,75
3,"Jan 4, 2021",2021-01-04,64
4,"Jan 5, 2021",2021-01-05,54
...,...,...,...
543,"Jun 26, 2022",2022-06-26,38
544,"Jun 27, 2022",2022-06-27,43
545,"Jun 28, 2022",2022-06-28,42
546,"Jun 29, 2022",2022-06-29,44


In [34]:
gt_filtered.to_csv('btc_gt_21_22.csv')

In [20]:
btc = pd.read_csv('Bitcoin Historical Data.csv', index_col=0)

In [21]:
# Calculate 24h% Change
btc['24h_change'] = btc['Close'].pct_change(periods=1) * 100
btc['24h_change'] = btc['24h_change'].apply(lambda x: f"{x:.2f}" if pd.isnull(x) or x >= 0 else f"{x:.2f}")

# Calculate 7d% Change
btc['7d_change'] = btc['Close'].pct_change(periods=7) * 100
btc['7d_change'] = btc['7d_change'].apply(lambda x: f"{x:.2f}" if pd.isnull(x) or x >= 0 else f"{x:.2f}")

# Calculate 30d% Change
btc['30d_change'] = btc['Close'].pct_change(periods=30) * 100
btc['30d_change'] = btc['30d_change'].apply(lambda x: f"{x:.2f}" if pd.isnull(x) or x >= 0 else f"{x:.2f}")

btc


,Date,Open,High,Low,Close,Adj Close,Volume,24h_change,7d_change,30d_change
0,2014-09-17,465.864014,468.174011,452.421997,457.334015,457.334015,21056800,nan,nan,nan
1,2014-09-18,456.859985,456.859985,413.104004,424.440002,424.440002,34483200,-7.19,nan,nan
2,2014-09-19,424.102997,427.834991,384.532013,394.795990,394.795990,37919700,-6.98,nan,nan
3,2014-09-20,394.673004,423.295990,389.882996,408.903992,408.903992,36863600,3.57,nan,nan
4,2014-09-21,408.084991,412.425995,393.181000,398.821014,398.821014,26580100,-2.47,nan,nan
...,...,...,...,...,...,...,...,...,...,...
3463,2024-03-11,69020.546875,72850.710938,67194.882813,72123.906250,72123.906250,65716656765,4.50,5.55,50.98
3464,2024-03-12,72125.125000,72825.656250,68728.851563,71481.289063,71481.289063,62554434520,-0.89,12.04,48.01
3465,2024-03-13,71482.117188,73637.476563,71334.093750,73083.500000,73083.500000,48212536929,2.24,10.55,46.29
3466,2024-03-14,73079.375000,73750.070313,68563.023438,71396.593750,71396.593750,59594605698,-2.31,6.68,43.53


In [22]:
# Calculate future percentage changes
btc['next_day_pct_change'] = btc['Close'].shift(-1).pct_change(periods=1) * 100
btc['next_7days_pct_change'] = btc['Close'].shift(-7).pct_change(periods=7) * 100
btc['next_30days_pct_change'] = btc['Close'].shift(-30).pct_change(periods=30) * 100


C:\Users\ASUS\AppData\Local\Temp\ipykernel_15472\338599435.py:2: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  btc['next_day_pct_change'] = btc['Close'].shift(-1).pct_change(periods=1) * 100
C:\Users\ASUS\AppData\Local\Temp\ipykernel_15472\338599435.py:3: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  btc['next_7days_pct_change'] = btc['Close'].shift(-7).pct_change(periods=7) * 100
C:\Users\ASUS\AppData\Local\Temp\ipykernel_15472\338599435.py:4: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading N

In [23]:
def classify_movement(change, threshold_up, threshold_down):
    if pd.isnull(change):
        return 0
    if change >= threshold_up:
        return 1
    elif change <= threshold_down:
        return -1
    else:
        return 0

# Apply the classification function
btc['next_1d_movement'] = btc['next_day_pct_change'].apply(lambda x: classify_movement(x, 2, -2))
btc['next_7d_movement'] = btc['next_7days_pct_change'].apply(lambda x: classify_movement(x, 5, -5))
btc['next_30d_movement'] = btc['next_30days_pct_change'].apply(lambda x: classify_movement(x, 10, -10))

# Drop intermediate percentage change columns
btc.drop(columns=['next_day_pct_change', 'next_7days_pct_change', 'next_30days_pct_change'], inplace=True)

# Display the DataFrame with new prediction columns
btc.head(20) # Display the first 20 rows to see more data

,Date,Open,High,Low,Close,Adj Close,Volume,24h_change,7d_change,30d_change,next_1d_movement,next_7d_movement,next_30d_movement
0,2014-09-17,465.864014,468.174011,452.421997,457.334015,457.334015,21056800,nan,nan,nan,0,0,0
1,2014-09-18,456.859985,456.859985,413.104004,424.440002,424.440002,34483200,-7.19,nan,nan,-1,0,0
2,2014-09-19,424.102997,427.834991,384.532013,394.795990,394.795990,37919700,-6.98,nan,nan,1,0,0
3,2014-09-20,394.673004,423.295990,389.882996,408.903992,408.903992,36863600,3.57,nan,nan,-1,0,0
4,2014-09-21,408.084991,412.425995,393.181000,398.821014,398.821014,26580100,-2.47,nan,nan,0,0,0
5,2014-09-22,399.100006,406.915985,397.130005,402.152008,402.152008,24127600,0.84,nan,nan,1,0,0
6,2014-09-23,402.092010,441.557007,396.196991,435.790985,435.790985,45099500,8.36,nan,nan,-1,0,0
7,2014-09-24,435.751007,436.112000,421.131989,423.204987,423.204987,30627700,-2.89,-7.46,nan,-1,-1,0
8,2014-09-25,423.156006,423.519989,409.467987,411.574005,411.574005,26814400,-2.75,-3.03,nan,0,-1,0
9,2014-09-26,411.428986,414.937988,400.009003,404.424988,404.424988,21460800,-1.74,2.44,nan,0,-1,0


In [24]:
btc.to_csv('Bitcoin Historical Data.csv')

In [25]:
# Filter the DataFrame to include only data between January 1, 2021, and June 30, 2022
btc_filter = btc[(btc['Date'] <= '2022-06-30') & (btc['Date'] >= '2021-01-01')]


# Reset the index of the resulting DataFrame if needed
btc_filter.reset_index(drop=True, inplace=True)

btc_filter = btc_filter.sort_values(by='Date').reset_index(drop=True)

In [26]:
btc_filter.to_csv('Bitcoin price 2021-2022.csv')

In [35]:
# Rename the 'date' column in btc_price to 'timestamp'
btc_filter.rename(columns={'Date': 'timestamp'}, inplace=True)

# Merge btc_price and filtered_data based on the 'timestamp' column
merged_df = pd.merge(btc_filter, filtered_data, how='inner', on='timestamp')

# Merge the resulting DataFrame with btc_gt based on the 'timestamp' column
merged_df = pd.merge(merged_df, gt_filtered, how='inner', on='timestamp')


In [36]:
merged_df.rename(columns={'value_x': 'fng_value',
                         'value_classification': 'fng_classification',
                         'value_y': 'btc_gt'}, inplace=True)

In [37]:
merged_df

,timestamp,Open,High,Low,Close,Adj Close,Volume,24h_change,7d_change,30d_change,next_1d_movement,next_7d_movement,next_30d_movement,fng_value,fng_classification,date,btc_gt
0,2021-01-01,28994.009766,29600.626953,28803.585938,29374.152344,29374.152344,40730301359,1.28,19.09,52.98,1,1,1,94,Extreme Greed,"Jan 1, 2021",39
1,2021-01-02,29376.455078,33155.117188,29091.181641,32127.267578,32127.267578,67865420765,9.37,21.52,65.22,1,1,0,94,Extreme Greed,"Jan 2, 2021",64
2,2021-01-03,32129.408203,34608.558594,32052.316406,32782.023438,32782.023438,78665235202,2.04,24.78,75.31,-1,1,0,93,Extreme Greed,"Jan 3, 2021",75
3,2021-01-04,32810.949219,33440.218750,28722.755859,31971.914063,31971.914063,81163475344,-2.47,18.04,66.92,1,1,1,94,Extreme Greed,"Jan 4, 2021",64
4,2021-01-05,31977.041016,34437.589844,30221.187500,33992.429688,33992.429688,67547324782,6.32,24.23,75.72,1,0,0,93,Extreme Greed,"Jan 5, 2021",54
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
543,2022-06-26,21496.494141,21783.724609,21016.269531,21027.294922,21027.294922,18027170497,-2.21,2.31,-26.55,0,-1,0,14,Extreme Fear,"Jun 26, 2022",38
544,2022-06-27,21028.238281,21478.089844,20620.199219,20735.478516,20735.478516,20965695707,-1.39,0.66,-28.04,-1,0,1,12,Extreme Fear,"Jun 27, 2022",43
545,2022-06-28,20731.544922,21164.423828,20228.812500,20280.634766,20280.634766,21381535161,-2.19,-2.08,-31.13,0,0,1,10,Extreme Fear,"Jun 28, 2022",42
546,2022-06-29,20281.169922,20364.156250,19937.791016,20104.023438,20104.023438,23552740328,-0.87,0.59,-36.63,0,0,1,13,Extreme Fear,"Jun 29, 2022",44


In [38]:
merged_df.drop(columns=['date'], inplace=True)

In [39]:
merged_df.rename(columns={'timestamp': 'date',
                         'Open': 'open',
                         'High': 'high',
                         'Low': 'low',
                         'Close': 'close',
                         'Adj Close': 'adj_close',
                         'Volume': 'volume'}, inplace=True)

In [40]:
merged_df.to_csv('combined_data.csv')

In [41]:
merged_df

,date,open,high,low,close,adj_close,volume,24h_change,7d_change,30d_change,next_1d_movement,next_7d_movement,next_30d_movement,fng_value,fng_classification,btc_gt
0,2021-01-01,28994.009766,29600.626953,28803.585938,29374.152344,29374.152344,40730301359,1.28,19.09,52.98,1,1,1,94,Extreme Greed,39
1,2021-01-02,29376.455078,33155.117188,29091.181641,32127.267578,32127.267578,67865420765,9.37,21.52,65.22,1,1,0,94,Extreme Greed,64
2,2021-01-03,32129.408203,34608.558594,32052.316406,32782.023438,32782.023438,78665235202,2.04,24.78,75.31,-1,1,0,93,Extreme Greed,75
3,2021-01-04,32810.949219,33440.218750,28722.755859,31971.914063,31971.914063,81163475344,-2.47,18.04,66.92,1,1,1,94,Extreme Greed,64
4,2021-01-05,31977.041016,34437.589844,30221.187500,33992.429688,33992.429688,67547324782,6.32,24.23,75.72,1,0,0,93,Extreme Greed,54
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
543,2022-06-26,21496.494141,21783.724609,21016.269531,21027.294922,21027.294922,18027170497,-2.21,2.31,-26.55,0,-1,0,14,Extreme Fear,38
544,2022-06-27,21028.238281,21478.089844,20620.199219,20735.478516,20735.478516,20965695707,-1.39,0.66,-28.04,-1,0,1,12,Extreme Fear,43
545,2022-06-28,20731.544922,21164.423828,20228.812500,20280.634766,20280.634766,21381535161,-2.19,-2.08,-31.13,0,0,1,10,Extreme Fear,42
546,2022-06-29,20281.169922,20364.156250,19937.791016,20104.023438,20104.023438,23552740328,-0.87,0.59,-36.63,0,0,1,13,Extreme Fear,44
